In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K
from keras import regularizers
import math

In [2]:
data = np.load("./images.npy")
label = np.load("./labels.npy")
data = data / 255.

In [3]:
ya_hour = 2 * np.pi * (label[:, 0] + label[:, 1] / 60) / 12
ya_hour1 = np.sin(ya_hour)
ya_hour2 = np.cos(ya_hour)
y_l = np.stack((ya_hour1, ya_hour2), axis=1)
y_l = np.concatenate((y_l, label), axis=1)

In [4]:
X, X_test, y, y_test = train_test_split(data, y_l, test_size=0.2, random_state=42)
print(f"Train set: {X.shape}")
print(f"Test set: {X_test.shape}")
y = y[:, :2]
y_test = y_test[:, 2:]

Train set: (14400, 150, 150)
Test set: (3600, 150, 150)


In [5]:
y_test = y_test[:, 0] + y_test[:, 1] / 60

In [6]:
K.clear_session()
final_model = keras.Sequential(
    [
        keras.Input(shape=(150,150,1)),
        layers.Conv2D(16, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.01),strides=1,kernel_regularizer=regularizers.L2(1e-4)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.01),strides=2,kernel_regularizer=regularizers.L2(1e-4)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.01),strides=1,kernel_regularizer=regularizers.L2(1e-4)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.1),
        layers.Dense(64,activation=keras.layers.LeakyReLU(alpha=0.01),kernel_regularizer=regularizers.L2(1e-4)),
        layers.Normalization(axis=None),
        layers.Dense(2,activation="tanh"),
    ]
)

final_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 16)      160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 36, 36, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 18, 18, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 64)         0

2022-11-21 07:22:08.130702: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 07:22:09.121571: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11424 MB memory:  -> device: 0, name: NVIDIA TITAN Xp, pci bus id: 0000:05:00.0, compute capability: 6.1


In [7]:
batch_size = 128
epochs = 300
final_model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0009,), metrics="mse")

In [8]:
checkpoint_filepath = "./best_model.hdf5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_mse',
    mode='min',
    save_best_only=True
)
final_model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[model_checkpoint_callback])

Epoch 1/300


2022-11-21 07:22:13.960250: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


102/102 [==============================] - 6s 33ms/step - loss: 0.5136 - mse: 0.5013 - val_loss: 0.5081 - val_mse: 0.4995
Epoch 2/300
102/102 [==============================] - 2s 24ms/step - loss: 0.4903 - mse: 0.4825 - val_loss: 0.4370 - val_mse: 0.4286
Epoch 3/300
102/102 [==============================] - 2s 24ms/step - loss: 0.3835 - mse: 0.3733 - val_loss: 0.3185 - val_mse: 0.3059
Epoch 4/300
102/102 [==============================] - 2s 23ms/step - loss: 0.2636 - mse: 0.2489 - val_loss: 0.2182 - val_mse: 0.2017
Epoch 5/300
102/102 [==============================] - 2s 23ms/step - loss: 0.1879 - mse: 0.1699 - val_loss: 0.1647 - val_mse: 0.1455
Epoch 6/300
102/102 [==============================] - 2s 23ms/step - loss: 0.1407 - mse: 0.1204 - val_loss: 0.1291 - val_mse: 0.1079
Epoch 7/300
102/102 [==============================] - 2s 24ms/step - loss: 0.1103 - mse: 0.0885 - val_loss: 0.1041 - val_mse: 0.0817
Epoch 8/300
102/102 [==============================] - 2s 23ms/step - loss

In [9]:
final_model.load_weights(checkpoint_filepath)
output=final_model.predict(X_test)
predict1=output[:,0]/np.sqrt(np.square(output[:,0])+np.square(output[:,1]))
predict2=output[:,1]/np.sqrt(np.square(output[:,0])+np.square(output[:,1]))

113/113 [==============================] - 1s 4ms/step


In [10]:
angle=[]
for i in range(len(predict1)):
    a_acos = math.acos(predict2[i])
    if predict1[i] < 0:
        angle.append( math.degrees(-a_acos) % 360 )
    else: 
        angle.append( math.degrees(a_acos) )

In [11]:
angle=np.stack(angle)
time_p=12*angle/360

def common_sense_reg(y_true, y_pred):
    d = np.abs(y_true-y_pred)
    return (1-(d//6))*(d)+(d//6)*(12-d)

common_sense_reg(y_test,time_p).mean()

0.19215443193802784